In [6]:
import os
from google.colab import drive
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from glob import glob
import sys

drive.mount('/content/drive/', force_remount=True)
PROJECT_ROOT_PATH = "/content/drive/MyDrive/Latitude_AI_Team"
os.chdir(PROJECT_ROOT_PATH)

CITYSCAPES_ROOT = os.path.join("data", "cityscapes")

CITYSCAPES_IMG_DIR = os.path.join(CITYSCAPES_ROOT, 'leftImg8bit', 'train')

CITYSCAPES_LABEL_DIR = os.path.join(CITYSCAPES_ROOT, 'gtFine', 'train')

print(f"Cityscapes Image Directory: {CITYSCAPES_IMG_DIR}")
print(f"Cityscapes Label Directory: {CITYSCAPES_LABEL_DIR}")


Mounted at /content/drive/
Cityscapes Image Directory: data/cityscapes/leftImg8bit/train
Cityscapes Label Directory: data/cityscapes/gtFine/train


In [8]:
#QAAAAAAAAAAA# --- Cityscapes Dataset Class ---
class CityscapesDataset(Dataset):
  def __init__(self, img_dir, label_dir, transforms=None, limit=None):
    self.img_dir = img_dir
    self.label_dir = label_dir
    self.transforms = transforms

    self.image_paths = sorted(glob(os.path.join(img_dir, '**', '*leftImg8bit.png'), recursive=True))
    self.label_paths = sorted(glob(os.path.join(label_dir, '**', '*gtFine_instanceIds.png'), recursive=True))

    if len(self.image_paths) != len(self.label_paths):
      raise ValueError("Number of images and labels do not match.")

    if limit is not None:
      self.image_paths = self.image_paths[:limit]
      self.label_paths = self.label_paths[:limit]

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    img = Image.open(self.image_paths[idx]).convert("RGB")
    instance_ids_img = Image.open(self.label_paths[idx])
    instance_ids_np = np.array(instance_ids_img, dtype=np.int32)

    obj_ids = np.unique(instance_ids_np)
    obj_ids = obj_ids[obj_ids > 10000]

    masks = []
    boxes = []
    labels = []

    for obj_id in obj_ids:
      mask_instance = (instance_ids_np == obj_id)
      if mask_instance.sum() < 5:
        continue

      pos = np.where(mask_instance)
      xmin = np.min(pos[1]); xmax = np.max(pos[1])
      ymin = np.min(pos[0]); ymax = np.max(pos[0])

      boxes.append([xmin, ymin, xmax, ymax])
      masks.append(mask_instance)

      labels.append(1)

    if boxes:
      boxes = torch.as_tensor(boxes, dtype=torch.float32)
      labels = torch.as_tensor(labels, dtype=torch.int64)
      masks = torch.as_tensor(np.stack(masks), dtype=torch.uint8)
    else:
      H, W = instance_ids_np.shape
      boxes = torch.zeros((0, 4), dtype=torch.float32)
      labels = torch.zeros((0,), dtype=torch.int64)
      masks = torch.zeros((0, H, W), dtype=torch.uint8)

    target = {
        "boxes": boxes,
        "labels": labels,
        "masks": masks,
        "image_id": torch.tensor([idx]),
    }

    if self.transforms:
      img = self.transforms(img)

    return img, target

  # Data Loader Function
def get_cityscapes_dataloader(img_dir, label_dir, batch_size=2, limit=None, shuffle=True):
  transform = transforms.Compose([transforms.ToTensor()])


  dataset = CityscapesDataset(
      img_dir=img_dir,
      label_dir=label_dir,
      transforms=transform,
      limit=limit
  )

  return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=lambda x: tuple(zip(*x))
    )


In [ ]:
#-----TEST ----------------
CITYSCAPES_TEST_LIMIT = 5 # small number for a fast test

try:
    cityscapes_test_loader = get_cityscapes_dataloader(
        img_dir=CITYSCAPES_IMG_DIR,
        label_dir=CITYSCAPES_LABEL_DIR,
        batch_size=1,
        limit=CITYSCAPES_TEST_LIMIT,
        shuffle=False
    )
    cityscapes_dataset = cityscapes_test_loader.dataset

    print(f"\nCityscapes Loader initialized with {len(cityscapes_dataset)} samples.")

    # Extract the first sample to verify the format
    images, targets = next(iter(cityscapes_test_loader))

    image = images[0]
    target = targets[0]
    num_objects = target['boxes'].shape[0]

    print("\n--- Cityscapes Data Format Verification ---")
    print(f"Image Tensor Shape: {image.shape}")
    print(f"Number of Objects Found: {num_objects}")
    print(f"Target Keys: {target.keys()}")
    print(f"Masks Shape: {target['masks'].shape} (Should be [N, H, W])")

    print("\n Cityscapes Data Loader is successfully defined and verified.")

except Exception as e:
    print(f"\n ERROR: Failed to load Cityscapes data. ")
    print(f"Check that files exist in: {CITYSCAPES_IMG_DIR}")
    print(f"Detailed Error: {e}")



Cityscapes Loader initialized with 5 samples.

--- Cityscapes Data Format Verification ---
Image Tensor Shape: torch.Size([3, 1024, 2048])
Number of Objects Found: 13
Target Keys: dict_keys(['boxes', 'labels', 'masks', 'image_id'])
Masks Shape: torch.Size([13, 1024, 2048]) (Should be [N, H, W])

 Cityscapes Data Loader is successfully defined and verified.
